In [3]:
import requests
import pandas as pd    
from datetime import datetime

base_url = 'https://npa.krungthai.com/api/v1'
url = base_url + '/product/searchAll'

payload = {
    "priceRangeMax": "20000000",
    "priceRangeMin": "10000",
    "shrProvince": "10",
    "typeProp": [
        "11"
    ],
    "paging": {
        "totalRows": 0,
        "rowsPerPage": 150,
        "currentPage": 1
    }
}

reqs = requests.post(url, verify=False, json=payload)
print(reqs.text)

df = pd.json_normalize(reqs.json()['dataResponse'])
print(df)

# df.to_csv(f"../../dataset/npa/krungthai_{datetime.today().strftime('%Y-%m-%d')}.csv")

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'npa.krungthai.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"dataResponse":[{"collGrpId":145721,"collId":null,"shrProvinceName":"กรุงเทพมหานคร","collTypeName":null,"calSumAreaCollGrpId":"63.3 (ตร.ม.)","fileName":"305645202304261550136509.jpg","isShow":"1","collDesc":null,"collCateName":"คอนโดมิเนียม/อาคารชุด","shrAmphurName":"เขตห้วยขวาง","nmlPrice":null,"price":"           3,368,000","priceNumber":"3368000","extraPrice":null,"collCode":"58BK021100171","collMonoCode":"68/34","shrTambonName":"แขวงสามเสนนอก","addr":"68/34 (2 ห้องนอน 2 ห้องน้ำ ชั้น 5 ) ดีไซน์ คอนโดมิเนียม รัชดา ชั้น 5 20 มิถุนา","priceStrDt":null,"priceEndDt":"31 พ.ค.  2567","collMonoCodeAll":"68/34","isSpeedy":0,"isNewAsset":null,"isPromotion":1,"isNewPay":0,"assetType":null,"area":"63.3 (ตร.ม.)","shrProvinceNo":null,"sumAreaNumt":63.3,"nplId":null,"assetDesc":null,"contactName":null,"contactTel":null,"lat":"13.788051","lon":"100.58299","orgCode":null,"useFor":null,"collTId":null,"refCode":null,"ledMktCollId":null,"listImg":null,"road":"สุทธิสารวินิจฉัย","cateName":"พร้อมขาย","c

# Upsert Database

In [28]:
import json
from mailjet_rest import Client
import os

api_key = "753737744b3bd671cfd55aae33f3c9e9"
api_secret = "34e5da49cc5032dcf81c362da1e1fd60"
mailjet = Client(auth=(api_key, api_secret), version='v3.1')

df = pd.json_normalize(reqs.json()['dataResponse'])
db_df = pd.read_csv("../../dataset/npa/krungthai_2023-11-01.csv")

df.set_index('collGrpId', inplace=True)
db_df.set_index('collGrpId', inplace=True)
db_df = db_df.drop(columns=df.columns)

property_detail_url = "https://npa.krungthai.com/propertyDetail/"

"""SOLD properties"""
# outer_join_df = db_df.join(df, lsuffix="_sold", how='outer')
# print(outer_join_df)

# sold_properties = outer_join_df[outer_join_df["shrProvinceName"].isna()]
# print(sold_properties)

# print(db_df.columns.size)

# sold_properties = sold_properties.drop(db_df, axis=1)
# print(sold_properties)

"""NEW properties"""
# outer_join_df = db_df.join(df, rsuffix="_new", how='outer')
# print(outer_join_df)

# sold_properties = outer_join_df[outer_join_df["shrProvinceName"].isna()]
# print(sold_properties)

# print(db_df.columns.size)

# sold_properties = sold_properties.drop(db_df, axis=1)
# print(sold_properties)

email_message = ""

"""DB UPDATE"""
inner_join_df = db_df.join(df, lsuffix="_db", how='inner')
for index, row in inner_join_df.iterrows():
    formatted_prices = [{
        "date": datetime.today().strftime('%Y-%m-%d'), "price": float(row['priceNumber'])
    }]
    for rawprice in json.loads(row['shrPriceVos_db']):
        if not rawprice in formatted_prices:
            formatted_prices.append(rawprice)
    inner_join_df.at[index,'shrPriceVos'] = json.dumps(formatted_prices)
    if float(row['priceNumber']) != float(row['priceNumber_db']):
        email_message += f"PRICE UPDATE - property {row['shareURL']} price update from {row['priceNumber_db']} to {row['priceNumber']}\r\n"

"""send email"""
if email_message != "":
    email_data = {
    'Messages': [
                    {
                            "From": {
                                    "Email": "bremard.alexandre@gmail.com",
                                    "Name": "RealDeal"
                            },
                            "To": [
                                    {
                                            "Email": "bremard.alexandre@gmail.com",
                                            "Name": "Alex"
                                    }
                            ],
                            "Subject": "NPA property price update",
                            "TextPart": email_message,
                    }
            ]
    }
    print(email_data)
    result = mailjet.send.create(data=email_data)
    print(result.status_code)
    print(result.json())

print(inner_join_df)
# inner_join_df.to_csv(f"../../dataset/npa/krungthai_{datetime.today().strftime('%Y-%m-%d')}.csv")

           Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  collId_db   
collGrpId                                                      
145721                0             0           0        NaN  \
146440                1             1           1        NaN   
146600                2             2           2        NaN   
169662                3             3           3        NaN   
175080                4             4           4        NaN   
...                 ...           ...         ...        ...   
236117              136           136         136        NaN   
239866              137           137         137        NaN   
239998              138           138         138        NaN   
243750              139           139         139        NaN   
243944              140           140         140        NaN   

          shrProvinceName_db  collTypeName_db calSumAreaCollGrpId_db   
collGrpId                                                              
145721         กรุงเทพม